<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_radiomic_and_dosimetric_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
This notebook demonstrates the extraction radiomic & dosimetric features using CERR.

### Requirements
* GNU Octave with `statistics`, `io` & `image` packages
* CERR
Note: Installing these tools may incur a one-time extra runtime overhead.

### I/O
Images and plan doses stored in CERR's native planC format (imported from DICOM) are used for feature calculation. Results are output to Octave structures/Python dictionaries.

---

## Installing dependencies
Uncomment (`Ctrl+/`) the following to install (1) Octave with required packages (2) Oct2py and (3) CERR.

### Install latest Octave from compile

In [1]:
# %%capture
# ! apt-get update
# ! apt-get install libgraphicsmagick++1-dev libsuitesparse-dev libqrupdate1 \
# libfftw3-3 gnuplot zsh openjdk-8-jdk

In [2]:
# %%capture
# # download compiled octave package (latest)
# def getOctave(root_path):
#   os.chdir(root_path)
#   with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
#       data = json.loads(url.read().decode())
#   fname = data['assets'][0]['name']
#   requrl = data['assets'][0]['browser_download_url']
#   urllib.request.urlretrieve(requrl, fname)
#   # unzip, identify octave folder name
#   !tar xf {fname}
#   top_folder = !tar tf {fname} | head -1
#   octave_folder = top_folder[0][:-1]
#   octave_path = os.path.join(root_path,octave_folder)
#   return octave_path

In [3]:
# %%capture
# import os, urllib.request, json
# octave_path = getOctave('/content')
os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'  #Replace with OCTAVE_EXECUTABLE path
os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']     #Replace with Octave path

###  Install Octatve-Python bridge

In [4]:
# %%capture
# ! pip3 install octave_kernel
# ! pip3 install oct2py==5.3.0


### Download CERR


In [5]:
# %%capture
# import os   
# !git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git
# currDir = os.getcwd()
# os.chdir("/content/CERR")
# !git checkout c2b65179da40622bc7b095f679edd17f5ebc681e
# os.chdir(currDir)

---

## Feature calculation

### Add Octave packages and CERR to path

In [6]:
%load_ext oct2py.ipython
from oct2py import octave

In [7]:
%%capture
%%octave
pkg load image
pkg load io
pkg load statistics

### Add CERR to Octave path

In [8]:
%%octave
%%capture
#Add CERR to path
cerrPath = '/content/CERR'
currDir = pwd;
cd(cerrPath)
addToPath2(cerrPath)
cd(currDir)

### Read radiomics calculation settings

In [9]:
%%capture
%%octave
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/head_neck_ex1_20may03.mat.bz2';
planC = loadPlanC(sampleData, tempdir);
planC = updatePlanFields(planC);
planC = quality_assure_planC(sampleData, planC);

sampleParam = '/content/CERR/Jupyter_Notebooks/demoParams.json';
paramS = getRadiomicsParamTemplate(sampleParam);

### Compute features

In [10]:
%%octave
indexS = planC{end};
strC = {planC{indexS.structures}.structureName};
strName = paramS.structuresC{1};
structNum = getMatchingIndex(strName,strC,'exact');
scanNum = getStructureAssociatedScan(structNum,planC);
doseNum = 1;
binWidth = 0.01;

structFieldName = ['struct_',repSpaceHyp(strName)];

In [11]:
%%capture
%%octave

# 1. Calc. radiomic features
radiomicsFeatS = calcGlobalRadiomicsFeatures...
(scanNum, structNum, paramS, planC);

# 2. Compute DVH-based metrics
[doseV,volV] = getDVH(structNum,doseNum,planC);
[doseBinsV,volHistV] = doseHist(doseV,volV,binWidth);

dvhFeatS.meanDose = calc_meanDose(doseBinsV, volHistV); # Mean dose
dvhFeatS.d10 = calc_Dx(doseBinsV, volHistV, 10, 1);     # D10cc
dvhFeatS.v30 = calc_Vx(doseBinsV, volHistV, 30, 0);     # V30Gy
dvhFeatS.MOH55 = calc_MOHx(doseBinsV, volHistV, 55);    # MOH55%

### Access computed features in Python 


In [12]:
from oct2py import octave

%octave_pull radiomicsFeatS dvhFeatS

# radiomicsFeatS and dvhFeatS are available as Python dictionaries:
print(radiomicsFeatS['shapeS'])
print(dvhFeatS)

{'majorAxis': 12.935351093633601, 'minorAxis': 8.351486075529538, 'leastAxis': 4.277901480411592, 'flatness': 0.33071398290202186, 'elongation': 0.6456327327396543, 'max3dDiameter': 10.898238182067871, 'max2dDiameterAxialPlane': 9.443092346191406, 'max2dDiameterSagittalPlane': 6.094661712646484, 'max2dDiameterCoronalPlane': 10.702075004577637, 'surfArea': 87.76143692711165, 'volume': 3.507840000000014, 'filledVolume': 3.507840000000014, 'Compactness1': 0.0024071837980390474, 'Compactness2': 0.002058831239949457, 'spherDisprop': 7.8606733612810356, 'sphericity': 0.12721556462651845, 'surfToVolRatio': 25.018654478856302}
{'meanDose': 46.96107504826258, 'd10': 66.865, 'v30': 32.515999999999984, 'MOH55': 55.1462897603486}
